In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define CIFAR-10 specific transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='data/', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='data/', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:03<00:00, 51.0MB/s]


Extracting data/cifar-10-python.tar.gz to data/
Files already downloaded and verified


In [ ]:

# Define the MaxPoolingCNN model
class MaxPoolingCNN(nn.Module):
    def __init__(self, num_classes=10):       ## cifar10_classes = ["Airplane","Automobile","Bird","Cat","Deer","Dog","Frog","Horse","Ship","Truck"]
        super(MaxPoolingCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Training function
def train(model, loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(loader):.4f}')

# Testing function
def test(model, loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / len(loader.dataset)
    return test_loss / len(loader), accuracy


# Helper function to plot loss and accuracy
def plot_metrics(losses, accuracies):
    epochs = range(1, len(losses) + 1)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, losses, '-o', label='Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracies, '-o', label='Accuracy', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training Accuracy')
    plt.legend()

    plt.show()

# Example Usage
if __name__ == "__main__":
    # Initialize model, loss and optimizer
    modelMax = MaxPoolingCNN(num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(modelMax.parameters(), lr=0.001)

    # Train the model
    train(modelMax, train_loader, criterion, optimizer, epochs=5)

    # Evaluate the model on test data
    test_loss, test_accuracy = test(modelMax, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


Epoch [1/5], Loss: 1.3707
Epoch [2/5], Loss: 0.9478
Epoch [3/5], Loss: 0.7484
Epoch [4/5], Loss: 0.6161
Epoch [5/5], Loss: 0.5018
Test Loss: 0.7463, Test Accuracy: 75.03%


In [ ]:


# Define AveragePoolingLayer
class AveragePoolingLayer(nn.Module):
    def __init__(self, pool_size=2):
        super(AveragePoolingLayer, self).__init__()
        self.pool = nn.AvgPool2d(kernel_size=pool_size, stride=pool_size)

    def forward(self, x):
        return self.pool(x)

# Define CNNWithAveragePooling
class CNNWithAveragePooling(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNWithAveragePooling, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # Adjusted for RGB input
        self.avg_pool = AveragePoolingLayer(pool_size=2)
        self.fc1 = nn.Linear(32 * 16 * 16, num_classes)  # Adjusted for CIFAR-10 image size

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)  # Flatten for FC layer
        x = self.fc1(x)
        return x

# Training function
def train(model, loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(loader):.4f}')

# Testing function
def test(model, loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    avg_loss = test_loss / len(loader)
    accuracy = 100 * correct / len(loader.dataset)
    print(f'Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_loss, accuracy  # Return both loss and accuracy

# Helper function to plot loss and accuracy
def plot_metrics(losses, accuracies):
    epochs = range(1, len(losses) + 1)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, losses, '-o', label='Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracies, '-o', label='Accuracy', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training Accuracy')
    plt.legend()

    plt.show()

# Example Usage
if __name__ == "__main__":

    # Initialize model, loss and optimizer
    modelAvg = CNNWithAveragePooling(num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(modelAvg.parameters(), lr=0.001)

    # Train the model
    train(modelAvg, train_loader, criterion, optimizer, epochs=5)

    # Evaluate the model on test data
    test_loss, test_accuracy = test(modelAvg, test_loader, criterion)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


Epoch [1/5], Loss: 1.4474
Epoch [2/5], Loss: 1.2018
Epoch [3/5], Loss: 1.1105
Epoch [4/5], Loss: 1.0372
Epoch [5/5], Loss: 0.9802
Test Loss: 1.0720, Test Accuracy: 62.94%
Test Loss: 1.0720, Test Accuracy: 62.94%


In [ ]:

# Define FuzzyPoolingLayer with Regularization
class FuzzyPoolingLayer(nn.Module):
    def __init__(self, pool_size=2, reg_lambda=0.01):
        super(FuzzyPoolingLayer, self).__init__()
        self.pool_size = pool_size
        self.reg_lambda = reg_lambda

    def gaussian_membership(self, x, mean, std):
        return torch.exp(-((x - mean) ** 2) / (2 * std ** 2))

    def regularization_penalty(self, gmf):
        penalty = torch.sum(gmf ** 2, dim=(2, 3))
        return self.reg_lambda * penalty.mean()

    def forward(self, x):
        batch_size, channels, height, width = x.size()
        output_height, output_width = height // self.pool_size, width // self.pool_size
        pooled_output = torch.zeros((batch_size, channels, output_height, output_width)).to(x.device)
        regp_total = 0

        for i in range(0, height, self.pool_size):
            for j in range(0, width, self.pool_size):
                window = x[:, :, i:i + self.pool_size, j:j + self.pool_size]
                mean = window.mean(dim=(2, 3), keepdim=True)
                std = window.std(dim=(2, 3), keepdim=True) + 1e-8

                gmf = self.gaussian_membership(window, mean, std)
                regp_total += self.regularization_penalty(gmf)

                threshold = torch.min(torch.max(gmf, dim=3)[0], dim=2)[0].unsqueeze(-1).unsqueeze(-1)
                dominant_features = window * (gmf >= threshold).float()
                weights = gmf / (gmf.sum(dim=(2, 3), keepdim=True) + 1e-8)
                pooled_value = (dominant_features * weights).sum(dim=(2, 3))
                pooled_output[:, :, i // self.pool_size, j // self.pool_size] = pooled_value

        self.regp_loss = regp_total / (batch_size * channels)
        return pooled_output

# Define FuzzyCNNWithRegP for CIFAR-10
class FuzzyCNNWithRegP(nn.Module):
    def __init__(self, num_classes=10):
        super(FuzzyCNNWithRegP, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # Adjusted for RGB input
        self.fuzzy_pool = FuzzyPoolingLayer(pool_size=2, reg_lambda=0.01)
        self.fc1 = nn.Linear(32 * 16 * 16, num_classes)  # Adjusted for CIFAR-10 image size

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.fuzzy_pool(x)
        x = x.view(x.size(0), -1)  # Flatten for FC layer
        x = self.fc1(x)
        return x

    def get_regp_loss(self):
        return self.fuzzy_pool.regp_loss

# Training function with RegP loss included
def train(model, loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        regp_loss_total = 0.0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            regp_loss = model.get_regp_loss()
            total_loss = loss + regp_loss  # Include RegP in total loss
            total_loss.backward()
            optimizer.step()

            running_loss += loss.item()
            regp_loss_total += regp_loss.item()
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(loader):.4f}, RegP Loss: {regp_loss_total / len(loader):.4f}')



# Testing function with return values
def test(model, loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / len(loader.dataset)
    print(f'Test Loss: {test_loss / len(loader):.4f}, Test Accuracy: {accuracy:.2f}%')

    return test_loss / len(loader), accuracy  # Return test_loss and accuracy




# Helper function to plot loss and accuracy
def plot_metrics(losses, accuracies):
    epochs = range(1, len(losses) + 1)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, losses, '-o', label='Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracies, '-o', label='Accuracy', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training Accuracy')
    plt.legend()

    plt.show()


# Example Usage
if __name__ == "__main__":
    # Initialize model, loss and optimizer
    modelFuzzy = FuzzyCNNWithRegP(num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(modelFuzzy.parameters(), lr=0.001)

    # Train the model
    train(modelFuzzy, train_loader, criterion, optimizer, epochs=3)

    # Evaluate the model on test data
    test_loss, test_accuracy = test(modelFuzzy, test_loader, criterion)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')



Epoch [1/3], Loss: 1.4774, RegP Loss: 0.0037
Epoch [2/3], Loss: 1.2268, RegP Loss: 0.0037
Epoch [3/3], Loss: 1.1414, RegP Loss: 0.0038
Test Loss: 1.2015, Test Accuracy: 57.07%
Test Loss: 1.2015, Test Accuracy: 57.07%


In [ ]:

# Define FuzzyPoolingLayer
class FuzzyPoolingLayer(nn.Module):
    def __init__(self, pool_size=2):
        super(FuzzyPoolingLayer, self).__init__()
        self.pool_size = pool_size

    def gaussian_membership(self, x, mean, std):
        return torch.exp(-((x - mean) ** 2) / (2 * std ** 2))

    def forward(self, x):
        batch_size, channels, height, width = x.size()
        output_height, output_width = height // self.pool_size, width // self.pool_size
        pooled_output = torch.zeros((batch_size, channels, output_height, output_width)).to(x.device)

        for i in range(0, height, self.pool_size):
            for j in range(0, width, self.pool_size):
                window = x[:, :, i:i + self.pool_size, j:j + self.pool_size]
                mean = window.mean(dim=(2, 3), keepdim=True)
                std = window.std(dim=(2, 3), keepdim=True) + 1e-8  # Avoid divide-by-zero
                gmf = self.gaussian_membership(window, mean, std)
                threshold = torch.min(torch.max(gmf, dim=3)[0], dim=2)[0].unsqueeze(-1).unsqueeze(-1)
                dominant_features = window * (gmf >= threshold).float()
                weights = gmf / (gmf.sum(dim=(2, 3), keepdim=True) + 1e-8)
                pooled_value = (dominant_features * weights).sum(dim=(2, 3))
                pooled_output[:, :, i // self.pool_size, j // self.pool_size] = pooled_value

        return pooled_output

# Define FuzzyCNN
class FuzzyCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(FuzzyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # Adjusted for RGB input
        self.fuzzy_pool = FuzzyPoolingLayer(pool_size=2)
        self.fc1 = nn.Linear(32 * 16 * 16, num_classes)  # Adjusted for CIFAR-10 image size

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.fuzzy_pool(x)
        x = x.view(x.size(0), -1)  # Flatten for FC layer
        x = self.fc1(x)
        return x


# Training function
def train(model, loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(loader):.4f}')


# Testing function with return values
def test(model, loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / len(loader.dataset)
    print(f'Test Loss: {test_loss / len(loader):.4f}, Test Accuracy: {accuracy:.2f}%')

    return test_loss / len(loader), accuracy  # Return test_loss and accuracy




# Helper function to plot loss and accuracy
def plot_metrics(losses, accuracies):
    epochs = range(1, len(losses) + 1)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, losses, '-o', label='Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracies, '-o', label='Accuracy', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training Accuracy')
    plt.legend()

    plt.show()

# Example Usage
if __name__ == "__main__":
    # Initialize model, loss and optimizer
    model = FuzzyCNN(num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    train(model, train_loader, criterion, optimizer, epochs=5)

    # Evaluate the model on test data
    test_loss, test_accuracy = test(model, test_loader, criterion)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

Epoch [1/5], Loss: 1.5010
Epoch [2/5], Loss: 1.2416
Epoch [3/5], Loss: 1.1658
Epoch [4/5], Loss: 1.1147
Epoch [5/5], Loss: 1.0721
Test Loss: 1.2000, Test Accuracy: 58.00%
Test Loss: 1.2000, Test Accuracy: 58.00%
